In [1]:
import scanpy as sc
import pandas as pd
import pytorch_lightning as pl

In [2]:
scdata = sc.read_h5ad("../data/raw/HCL_final_USE.h5ad", backed="r")
n, _ = scdata.shape

In [3]:
i1 = int(n * 0.8)
i2 = int(n * 0.9)

In [ ]:
for split_type, i, j in [("train", None, i1),
                         ("test", i1, i2),
                         ("val", i2, None)]:
    scdata[i:j,:].write_h5ad(f"hcl_{split_type}_data.h5ad")

/Users/jeremyfisher/miniconda3/envs/scanpy/lib/python3.8/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


In [4]:
scdata = sc.read_h5ad("/Users/jeremyfisher/Desktop/02-712-project/data/intermediate/hcl_val_data.h5ad", backed="r")

In [5]:
scdata.shape

(479941, 3113)

In [ ]:
class SingleCellDataset:
    def __init__(self, anndata_fp):
        self.annotations = sc.read_h5ad(anndata_fp)
        self.genes = [str(gene) for gene in scdata.var_names.tolist()]
        
    def __len__(self):
        n, _ = self.annotations.shape
        return n
    
    def __getitem__(self, idx):
        gene_expression = self.annotations.X[idx,:]
        cell_type = str(self.annotations.obs.tissue)
        return gene_expression, cell_type
    
    @property
    def X(self):
        return self.annotations.X
    
    @property
    def y(self):
        return self.obs.tissue

In [ ]:
class SingleCellDataModule(pl.LightningDataModule):
    def __init__(self, train_fp, test_fp, val_fp, batch_size):
        super().__init__()
        self.train_fp = train_fp
        self.test_fp = test_fp
        self.val_fp = val_fp
        self.batch_size = batch_size

    def setup(self, stage=None):
        self.train_dataset = SingleCellDataset(self.train_fp)
        self.test_dataset = SingleCellDataset(self.test_fp)
        self.val_dataset = SingleCellDataset(self.val_fp)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)